In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, cohen_kappa_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from IPython.display import display


# --- Import your custom model files ---
from mop_model import MoPModel as MoP_raw, MoPConfig

def train_and_evaluate_model(model_name, model, X_train, y_train, X_val, y_val, X_test, y_test):
    """
    Trains a PyTorch model on the full feature set and evaluates its performance.
    """
    print(f"\n{'='*25}\n🚀 Training Model: {model_name}\n{'='*25}")
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    
    # --- Data Preparation ---
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    # Convert to Tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.LongTensor(y_train.values if isinstance(y_train, pd.Series) else y_train).to(device)
    X_val_tensor = torch.FloatTensor(X_val_scaled).to(device)
    y_val_tensor = torch.LongTensor(y_val.values if isinstance(y_val, pd.Series) else y_val).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.LongTensor(y_test.values if isinstance(y_test, pd.Series) else y_test).to(device)

    # --- Training ---
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    epochs = 25
    batch_size = 32

    dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for X_batch, y_batch in loader:
            optimizer.zero_grad()
            
            # MoP model requires a sequence dimension
            y_pred, _, _ = model(X_batch.unsqueeze(1))
            y_pred = y_pred.squeeze(1)

            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        if (epoch + 1) % 5 == 0:
            print(f"  Epoch {epoch+1}/{epochs}, Avg Loss: {epoch_loss/len(loader):.4f}")

    # --- Evaluation ---
    model.eval()
    results = []
    
    for name, X_tensor, y_tensor in [('Validation', X_val_tensor, y_val_tensor), 
                                     ('Test', X_test_tensor, y_test_tensor)]:
        with torch.no_grad():
            y_pred_tensor, _, _ = model(X_tensor.unsqueeze(1))
            y_pred_tensor = y_pred_tensor.squeeze(1)
            
            probas = nn.functional.softmax(y_pred_tensor, dim=1)
            _, predicted = torch.max(probas, 1)
            
            y_true = y_tensor.cpu().numpy()
            y_pred = predicted.cpu().numpy()
            y_score = probas[:, 1].cpu().numpy()

            print(f"\n--- Performance on {name} Set for {model_name} ---")
            print(classification_report(y_true, y_pred, zero_division=0))
            results.append({
                'Model': model_name,
                'Dataset': name,
                'Accuracy': accuracy_score(y_true, y_pred),
                'ROC-AUC': roc_auc_score(y_true, y_score),
                'Cohen Kappa': cohen_kappa_score(y_true, y_pred)
            })
            
    return pd.DataFrame(results)

# ===================================================================
# Main Execution Block for MoP
# ===================================================================

if __name__ == '__main__':
    try:
        print("🔹 Loading and preparing data for MoP evaluation...")
        df = pd.read_csv('uk_biobank_dataset.csv', low_memory=False)
        
        X = df.drop(columns=['Dementia Status'])
        y = df['Dementia Status']
        
        X_train, X_temp, y_train, y_temp = train_test_split(
            X, y, test_size=0.3, random_state=42, stratify=y)
        X_val, X_test, y_val, y_test = train_test_split(
            X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

        print(f"Data loaded successfully. Train shape: {X_train.shape}")
        n_features = X_train.shape[1]

    except FileNotFoundError:
        print("\n⚠️ ERROR: Data file not found. Please update the path.")
        exit()

    # --- Configure and Run MoP Evaluation ---
    mop_config = MoPConfig(
        input_dim=n_features, output_dim=2, intermediate_dim=64,
        layers=["0,8,16"], task_id="dementia_task"
    )
    mop_model_instance = MoP_raw(mop_config)

    mop_results = train_and_evaluate_model(
        'MoP', mop_model_instance, X_train, y_train, X_val, y_val, X_test, y_test
    )
    
    print("\n\n" + "="*25 + "\n✅ FINAL MoP SUMMARY\n" + "="*25)
    display(mop_results)


🔹 Loading and preparing data for MoP evaluation...
Data loaded successfully. Train shape: (1297, 64)

🚀 Training Model: MoP
  Epoch 5/25, Avg Loss: 0.4697
  Epoch 10/25, Avg Loss: 0.3822
  Epoch 15/25, Avg Loss: 0.2637
  Epoch 20/25, Avg Loss: 0.1818
  Epoch 25/25, Avg Loss: 0.1535

--- Performance on Validation Set for MoP ---
              precision    recall  f1-score   support

           0       0.65      0.64      0.64       137
           1       0.65      0.67      0.66       141

    accuracy                           0.65       278
   macro avg       0.65      0.65      0.65       278
weighted avg       0.65      0.65      0.65       278


--- Performance on Test Set for MoP ---
              precision    recall  f1-score   support

           0       0.71      0.72      0.72       138
           1       0.72      0.71      0.71       140

    accuracy                           0.72       278
   macro avg       0.72      0.72      0.72       278
weighted avg       0.72      0

,Model,Dataset,Accuracy,ROC-AUC,Cohen Kappa
0,MoP,Validation,0.651079,0.717762,0.301797
1,MoP,Test,0.715827,0.784110,0.431714


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, cohen_kappa_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from IPython.display import display

# --- Import your custom model files ---
from moe_model import MoE as MoE_raw, MLP as MoE_Expert

# --- Fix for MoE model to output logits ---
class MoE_Expert_Logits(MoE_Expert):
    def __init__(self, input_size, output_size, hidden_size):
        super().__init__(input_size, output_size, hidden_size)
        self.soft = nn.Identity()

def train_and_evaluate_model(model_name, model, X_train, y_train, X_val, y_val, X_test, y_test):
    """
    Trains a PyTorch model on the full feature set and evaluates its performance.
    """
    print(f"\n{'='*25}\n🚀 Training Model: {model_name}\n{'='*25}")
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    
    # --- Data Preparation ---
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    # Convert to Tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.LongTensor(y_train.values if isinstance(y_train, pd.Series) else y_train).to(device)
    X_val_tensor = torch.FloatTensor(X_val_scaled).to(device)
    y_val_tensor = torch.LongTensor(y_val.values if isinstance(y_val, pd.Series) else y_val).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.LongTensor(y_test.values if isinstance(y_test, pd.Series) else y_test).to(device)

    # --- Training ---
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    epochs = 25
    batch_size = 32

    dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for X_batch, y_batch in loader:
            optimizer.zero_grad()
            
            y_pred, aux_loss = model(X_batch)

            loss = criterion(y_pred, y_batch)
            loss += aux_loss # Add the load balancing loss

            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        if (epoch + 1) % 5 == 0:
            print(f"  Epoch {epoch+1}/{epochs}, Avg Loss: {epoch_loss/len(loader):.4f}")

    # --- Evaluation ---
    model.eval()
    results = []
    
    for name, X_tensor, y_tensor in [('Validation', X_val_tensor, y_val_tensor), 
                                     ('Test', X_test_tensor, y_test_tensor)]:
        with torch.no_grad():
            y_pred_tensor, _ = model(X_tensor)
            
            probas = nn.functional.softmax(y_pred_tensor, dim=1)
            _, predicted = torch.max(probas, 1)
            
            y_true = y_tensor.cpu().numpy()
            y_pred = predicted.cpu().numpy()
            y_score = probas[:, 1].cpu().numpy()

            print(f"\n--- Performance on {name} Set for {model_name} ---")
            print(classification_report(y_true, y_pred, zero_division=0))
            results.append({
                'Model': model_name,
                'Dataset': name,
                'Accuracy': accuracy_score(y_true, y_pred),
                'ROC-AUC': roc_auc_score(y_true, y_score),
                'Cohen Kappa': cohen_kappa_score(y_true, y_pred)
            })
            
    return pd.DataFrame(results)

# ===================================================================
# Main Execution Block for MoE
# ===================================================================

if __name__ == '__main__':
    try:
        print("🔹 Loading and preparing data for MoE evaluation...")
        df = pd.read_csv('uk_biobank_dataset.csv', low_memory=False)
        
        X = df.drop(columns=['Dementia Status'])
        y = df['Dementia Status']
        
        X_train, X_temp, y_train, y_temp = train_test_split(
            X, y, test_size=0.3, random_state=42, stratify=y)
        X_val, X_test, y_val, y_test = train_test_split(
            X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

        print(f"Data loaded successfully. Train shape: {X_train.shape}")
        n_features = X_train.shape[1]

    except FileNotFoundError:
        print("\n⚠️ ERROR: Data file not found. Please update the path.")
        exit()

    # --- Configure and Run MoE Evaluation ---
    moe_model_instance = MoE_raw(
        input_size=n_features, output_size=2, num_experts=4,
        hidden_size=16, k=2
    )
    # Replace experts with the Logits version
    moe_model_instance.experts = nn.ModuleList([
        MoE_Expert_Logits(
            input_size=n_features,
            output_size=2,
            hidden_size=16 
        ) for _ in range(moe_model_instance.num_experts)
    ])

    moe_results = train_and_evaluate_model(
        'MoE', moe_model_instance, X_train, y_train, X_val, y_val, X_test, y_test
    )
    
    print("\n\n" + "="*25 + "\n✅ FINAL MoE SUMMARY\n" + "="*25)
    display(moe_results)


🔹 Loading and preparing data for MoE evaluation...
Data loaded successfully. Train shape: (1297, 64)

🚀 Training Model: MoE
  Epoch 5/25, Avg Loss: 0.5067
  Epoch 10/25, Avg Loss: 0.4521
  Epoch 15/25, Avg Loss: 0.4262
  Epoch 20/25, Avg Loss: 0.3917
  Epoch 25/25, Avg Loss: 0.3543

--- Performance on Validation Set for MoE ---
              precision    recall  f1-score   support

           0       0.71      0.65      0.68       137
           1       0.68      0.74      0.71       141

    accuracy                           0.69       278
   macro avg       0.70      0.69      0.69       278
weighted avg       0.70      0.69      0.69       278


--- Performance on Test Set for MoE ---
              precision    recall  f1-score   support

           0       0.74      0.75      0.74       138
           1       0.75      0.74      0.74       140

    accuracy                           0.74       278
   macro avg       0.74      0.74      0.74       278
weighted avg       0.74      0

,Model,Dataset,Accuracy,ROC-AUC,Cohen Kappa
0,MoE,Validation,0.694245,0.774292,0.387665
1,MoE,Test,0.741007,0.825362,0.482041
